In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import gmplot
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time


In [67]:
df  = pd.read_excel("Business_Dataset.xlsx")
df = df.drop(['tbl', 'Geocoded_City1', 'Geocoded_City2'], axis=1)
pd.to_pickle(df, 'df_pickle.pkl')

KeyboardInterrupt: 

In [68]:
df = pd.read_pickle('df_pickle.pkl')

In [76]:
print(len(df))
df.head(5)

245955


,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk
0,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,970,180,81.43,G4,1.0000,81.43,G4,1.0000,81.43,202131013514112ABEPIE
1,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,TPA,970,19,208.93,DL,0.4659,219.98,UA,0.1193,154.11,202131013515304ABETPA
2,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,DAL,580,204,184.56,WN,0.9968,184.44,WN,0.9968,184.44,202131014011259ABQDAL
3,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,DFW,580,264,182.64,AA,0.9774,183.09,AA,0.9774,183.09,202131014011298ABQDFW
4,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,PHX,328,398,177.11,WN,0.6061,184.49,AA,0.3939,165.77,202131014014107ABQPHX


In [77]:
df['city1_small'] = df['city1'].str.split('/').str[0].str.split(',').str[0]
df['city2_small'] = df['city2'].str.split('/').str[0].str.split(',').str[0]
unique_city1 = list(set(df['city1']))
unique_city2 = list(set(df['city2']))
df.head()

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk,city1_small,city2_small
0,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,202131013514112ABEPIE,Allentown,Tampa
1,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,TPA,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,202131013515304ABETPA,Allentown,Tampa
2,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,DAL,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,202131014011259ABQDAL,Albuquerque,Dallas
3,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,DFW,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,202131014011298ABQDFW,Albuquerque,Dallas
4,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,PHX,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,202131014014107ABQPHX,Albuquerque,Phoenix


In [78]:
df['city1_small'] = df['city1'].str.split('/').str[0].str.split(',').str[0]
df['city2_small'] = df['city2'].str.split('/').str[0].str.split(',').str[0]

unique_city1 = list(set(df['city1_small']))
unique_city2 = list(set(df['city2_small']))

all_unique_cities = set(unique_city1) | set(unique_city2)
all_unique_cities

{'Albany',
 'Albuquerque',
 'Allentown',
 'Amarillo',
 'Appleton',
 'Asheville',
 'Ashland',
 'Aspen',
 'Atlanta',
 'Atlantic City',
 'Augusta',
 'Austin',
 'Bangor',
 'Baton Rouge',
 'Belleville',
 'Bellingham',
 'Bend',
 'Billings',
 'Birmingham',
 'Bismarck',
 'Bloomington',
 'Boise',
 'Boston',
 'Bozeman',
 'Buffalo',
 'Bullhead City',
 'Burlington',
 'Cedar Rapids',
 'Charleston',
 'Charlotte',
 'Charlottesville',
 'Chattanooga',
 'Chicago',
 'Cincinnati',
 'Cleveland',
 'Colorado Springs',
 'Columbia',
 'Columbus',
 'Concord',
 'Corpus Christi',
 'Dallas',
 'Dayton',
 'Daytona Beach',
 'Denver',
 'Des Moines',
 'Detroit',
 'Eagle',
 'El Paso',
 'Eugene',
 'Eureka',
 'Evansville',
 'Everett',
 'Fargo',
 'Fayetteville',
 'Flint',
 'Fort Collins',
 'Fort Myers',
 'Fort Wayne',
 'Fresno',
 'Grand Forks',
 'Grand Rapids',
 'Greensboro',
 'Greenville',
 'Gulfport',
 'Harlingen',
 'Harrisburg',
 'Hartford',
 'Hilton Head',
 'Houston',
 'Huntsville',
 'Idaho Falls',
 'Indianapolis',
 'Ja

In [46]:
geolocator = Nominatim(user_agent="city-density-map")
city_long = {}
city_lati = {}

# Function to get latitude and longitude
import time

def get_lat_long(city, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(city + ", USA", timeout=10)
            if location:
                return location.latitude, location.longitude
        except Exception as e:
            print(f"Attempt {attempt + 1}/{retries} failed: {e}")
            time.sleep(2)
    return None, None

for i, city in enumerate(all_unique_cities):
    print(f"Processing {i + 1}/{len(all_unique_cities)}: {city}")
    lat, long = get_lat_long(city)
    if lat is not None and long is not None:
        city_lati[city] = lat
        city_long[city] = long
    time.sleep(2)


Processing 1/168: Oklahoma City
Processing 2/168: Panama City
Processing 3/168: Plattsburgh
Processing 4/168: Salinas
Processing 5/168: Bangor
Processing 6/168: Wilmington
Processing 7/168: San Antonio
Processing 8/168: Phoenix
Processing 9/168: Fargo
Processing 10/168: Tallahassee
Processing 11/168: Pasco
Processing 12/168: Gulfport
Processing 13/168: Las Vegas
Processing 14/168: Santa Rosa
Processing 15/168: New Orleans
Processing 16/168: Ashland
Processing 17/168: Pensacola
Processing 18/168: Portland
Processing 19/168: Fort Myers
Processing 20/168: Sacramento
Processing 21/168: Kalispell
Processing 22/168: El Paso
Processing 23/168: Cedar Rapids
Processing 24/168: Harrisburg
Processing 25/168: Provo
Processing 26/168: Sioux Falls
Processing 27/168: Valparaiso
Processing 28/168: Albuquerque
Processing 29/168: Allentown
Processing 30/168: Montgomery
Processing 31/168: Austin
Processing 32/168: Roanoke
Processing 33/168: Traverse City
Processing 34/168: Jacksonville
Processing 35/168:

In [79]:
df['city1_lati'] = df['city1_small'].map(city_lati)
df['city1_long'] = df['city1_small'].map(city_long)

df['city2_lati'] = df['city2_small'].map(city_lati)
df['city2_long'] = df['city2_small'].map(city_long)

In [80]:
pd.to_pickle(df, 'df_pickle.pkl')

In [81]:
df.isnull().sum()

Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
airportid_1          0
airportid_2          0
airport_1            0
airport_2            0
nsmiles              0
passengers           0
fare                 0
carrier_lg        1540
large_ms          1540
fare_lg           1540
carrier_low       1612
lf_ms             1612
fare_low          1612
tbl1apk              0
city1_small          0
city2_small          0
city1_lati           0
city1_long           0
city2_lati           0
city2_long           0
dtype: int64

In [82]:
df.dropna(subset=['carrier_lg', 'large_ms', 'fare_lg', 'carrier_low', 'lf_ms', 'fare_low'], inplace=True)
df.head()

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,...,carrier_low,lf_ms,fare_low,tbl1apk,city1_small,city2_small,city1_lati,city1_long,city2_lati,city2_long
0,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,...,G4,1.0000,81.43,202131013514112ABEPIE,Allentown,Tampa,40.602206,-75.471279,27.947760,-82.458444
1,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,TPA,...,UA,0.1193,154.11,202131013515304ABETPA,Allentown,Tampa,40.602206,-75.471279,27.947760,-82.458444
2,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,DAL,...,WN,0.9968,184.44,202131014011259ABQDAL,Albuquerque,Dallas,35.084103,-106.650985,32.776272,-96.796856
3,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,DFW,...,AA,0.9774,183.09,202131014011298ABQDFW,Albuquerque,Dallas,35.084103,-106.650985,32.776272,-96.796856
4,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,PHX,...,AA,0.3939,165.77,202131014014107ABQPHX,Albuquerque,Phoenix,35.084103,-106.650985,33.448437,-112.074141


In [83]:
#Finding cardinatiy for data 
cardinality ={}
for col in df.columns:
    cardinality[col] = df[col].nunique()
cardinality

{'Year': 31,
 'quarter': 4,
 'citymarketid_1': 141,
 'citymarketid_2': 128,
 'city1': 141,
 'city2': 128,
 'airportid_1': 184,
 'airportid_2': 170,
 'airport_1': 183,
 'airport_2': 170,
 'nsmiles': 1155,
 'passengers': 3883,
 'fare': 36082,
 'carrier_lg': 67,
 'large_ms': 7352,
 'fare_lg': 37504,
 'carrier_low': 71,
 'lf_ms': 9687,
 'fare_low': 32283,
 'tbl1apk': 244343,
 'city1_small': 139,
 'city2_small': 125,
 'city1_lati': 139,
 'city1_long': 139,
 'city2_lati': 125,
 'city2_long': 125}

In [84]:
value_counts = {}

for col in df.columns:
    for value in df[col]:
        if value in value_counts:
            value_counts[value] += 1
        else:
            value_counts[value] = 1

value_counts_df = pd.DataFrame(list(value_counts.items()), columns=['Value', 'Count'])

value_counts_df

,Value,Count
0,2021,8181
1,2022,8095
2,2010,8101
3,1998,8843
4,2009,8095
...,...,...
300006,-89.093966,2
300007,-82.051206,2
300008,-108.240386,2
300009,-94.1642,1


In [85]:
df['Year'] = df['Year'] - df['Year'].min()

In [86]:
df = pd.get_dummies(df, columns=['quarter'])

In [87]:
#New dataframe where airportid_1 and 
# airportid_2 are replaced by their target-encoded versions based on fare_low.

airportid_1_encoded = df.groupby('airportid_1')['fare_low'].mean().to_dict()
df['airportid_1_encoded'] = df['airportid_1'].map(airportid_1_encoded)
airportid_2_encoded = df.groupby('airportid_2')['fare_low'].mean().to_dict()
df['airportid_2_encoded'] = df['airportid_2'].map(airportid_2_encoded)
df.drop(['airportid_1', 'airportid_2'], axis=1, inplace=True)
df.head()

,Year,citymarketid_1,citymarketid_2,city1,city2,airport_1,airport_2,nsmiles,passengers,fare,...,city1_lati,city1_long,city2_lati,city2_long,quarter_1,quarter_2,quarter_3,quarter_4,airportid_1_encoded,airportid_2_encoded
0,28,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",ABE,PIE,970,180,81.43,...,40.602206,-75.471279,27.947760,-82.458444,False,False,True,False,191.835909,129.117099
1,28,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",ABE,TPA,970,19,208.93,...,40.602206,-75.471279,27.947760,-82.458444,False,False,True,False,191.835909,156.790121
2,28,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",ABQ,DAL,580,204,184.56,...,35.084103,-106.650985,32.776272,-96.796856,False,False,True,False,188.713407,182.397671
3,28,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",ABQ,DFW,580,264,182.64,...,35.084103,-106.650985,32.776272,-96.796856,False,False,True,False,188.713407,186.393335
4,28,30140,30466,"Albuquerque, NM","Phoenix, AZ",ABQ,PHX,328,398,177.11,...,35.084103,-106.650985,33.448437,-112.074141,False,False,True,False,188.713407,185.221374
